In [25]:
import tensorflow as tf
from tensorflow import keras
from keras import regularizers
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Activation, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator

import os

In [26]:
import numpy as np 
import matplotlib.pyplot as plt
from PIL import Image

In [27]:
TARGET_SIZE = (100, 100)

## Data Augmentation 

In [28]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.2, 1.5]
)

### Generator of train images

In [29]:
data = os.path.join(os.getcwd(), 'dataset/mini_split/train')

train_generator = datagen.flow_from_directory(   
    data, 
    target_size=TARGET_SIZE,
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale'
)

num_batches = len(train_generator)
print("Number of batches: ", num_batches)

Found 400 images belonging to 5 classes.
Number of batches:  13


### Generator of test images

In [30]:
data = os.path.join(os.getcwd(), 'dataset/mini_split/test')

test_generator = datagen.flow_from_directory(
    data, 
    target_size=TARGET_SIZE,
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale'
)

num_batches = len(test_generator)
print("Number of batches: ", num_batches)

Found 100 images belonging to 5 classes.
Number of batches:  4


Son 100 imagenes, de 100x100, en los 3 canales RGB.

## Modelo de Datos

In [31]:
base_filters = 32
w_regularizer = 1e-4

In [32]:
model = tf.keras.Sequential()

# First layer
model.add(Conv2D(filters=base_filters, kernel_size=(3,3), kernel_regularizer=regularizers.l2(w_regularizer), padding='same', activation='relu', input_shape=(100,100,1)))
model.add(Activation('relu'))
model.add(BatchNormalization())

# Second layer
model.add(Conv2D(filters=2*base_filters, kernel_size=(3,3), kernel_regularizer=regularizers.l2(w_regularizer), padding='same', activation='relu'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))

# Third layer
model.add(Conv2D(filters=2*base_filters, kernel_size=(3,3), kernel_regularizer=regularizers.l2(w_regularizer), padding='same', activation='relu'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

# Fourth layer
model.add(Conv2D(filters=2*base_filters, kernel_size=(3,3), kernel_regularizer=regularizers.l2(w_regularizer), padding='same', activation='relu'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

# Fifth layer
model.add(Conv2D(filters=3*base_filters, kernel_size=(3,3), kernel_regularizer=regularizers.l2(w_regularizer), padding='same', activation='relu'))
model.add(Activation('relu'))
model.add(BatchNormalization())

# Sixth layer
model.add(Conv2D(filters=3*base_filters, kernel_size=(3,3), kernel_regularizer=regularizers.l2(w_regularizer), padding='same', activation='relu'))
model.add(Activation('relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))

# Flatten the output
model.add(Flatten())
model.add(Dense(5, activation='softmax'))

model.summary()


Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_12 (Conv2D)          (None, 100, 100, 32)      320       
                                                                 
 activation_12 (Activation)  (None, 100, 100, 32)      0         
                                                                 
 batch_normalization_12 (Bat  (None, 100, 100, 32)     128       
 chNormalization)                                                
                                                                 
 conv2d_13 (Conv2D)          (None, 100, 100, 64)      18496     
                                                                 
 activation_13 (Activation)  (None, 100, 100, 64)      0         
                                                                 
 batch_normalization_13 (Bat  (None, 100, 100, 64)     256       
 chNormalization)                                     

In [33]:
model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [34]:
hist = model.fit(train_generator, steps_per_epoch=len(train_generator), epochs=100, verbose=1)

Epoch 1/100
13/13 [==============================] - 1s 37ms/step - loss: 11.1076 - accuracy: 0.2425
Epoch 2/100
13/13 [==============================] - 0s 36ms/step - loss: 5.7975 - accuracy: 0.2775
Epoch 3/100
13/13 [==============================] - 0s 37ms/step - loss: 6.6965 - accuracy: 0.2775
Epoch 4/100
13/13 [==============================] - 0s 36ms/step - loss: 5.6024 - accuracy: 0.2800
Epoch 5/100
13/13 [==============================] - 0s 36ms/step - loss: 5.2096 - accuracy: 0.2825
Epoch 6/100
13/13 [==============================] - 0s 36ms/step - loss: 4.5246 - accuracy: 0.2950
Epoch 7/100
13/13 [==============================] - 1s 37ms/step - loss: 4.5557 - accuracy: 0.2900
Epoch 8/100
13/13 [==============================] - 1s 37ms/step - loss: 4.3500 - accuracy: 0.3475
Epoch 9/100
13/13 [==============================] - 1s 37ms/step - loss: 4.5826 - accuracy: 0.3600
Epoch 10/100
13/13 [==============================] - 1s 37ms/step - loss: 4.2782 - accuracy: 0.315

In [35]:
# Assuming test_images and test_labels are your testing data and labels
test_loss, test_acc = model.evaluate(test_generator, verbose=1)

print('\nTest accuracy:', test_acc)
print('\nTest loss:', test_loss)

4/4 [==============================] - 0s 19ms/step - loss: 1.3420 - accuracy: 0.6300

Test accuracy: 0.6299999952316284

Test loss: 1.3420087099075317


## Save model 

In [36]:
# Save the model
model.save('out/cnn2.keras')